CFDoRF - Exercise 2: Finite difference method

We'll be solving a simple advection - diffusion equation in 2D: ft + u°fx + v°fy == D°(fxx + fyy)

Setup and Data:

In [1]:
# Libraries:
from matplotlib import pyplot as plt
from matplotlib import cm, animation
from IPython.display import HTML
import timeit
import numpy as np

# Start timing:
start = timeit.default_timer()

#Constants:
D = 0.025; #[m^2/s] Diffusivity
u = -1; # velocity along x
v = 0; # velocity along y

# Discretization along x (independant):
NPx = 33 # Number of points
Lx = 2.0 # Length of the domain
dx = Lx/(NPx-1) # Step

# Discretization along y(independant) :
NPy = 33 # Number of points
Ly = 1.5 # Length of the domain
dy = Ly/(NPy-1) # Step

# Discretization along t(dependant - conditions from stab. analysis): :
NT = 150 #[-] Number of nodes
dt_diff = dx*dy/D/4 #[m] Diffusion limit condition for dt
dt_conv = 4*D/np.sqrt(u**2+v**2) #[m] Convection limit condition for dt
cs = 0.75 #[-] Safety coefficient
dt = cs*np.amin(np.array([dt_diff, dt_conv])) #[s] Time step

# Discretized domain:
xspan = np.linspace(0,Lx,NPx)
yspan = np.linspace(0,Ly,NPy)
tspan = np.linspace(0,NT*dt,NT)

# 2D Parabolic Boundary - Sides - Boundary conditions:
rangeE = range(int(np.floor(NPy*1/3)),int(np.ceil(NPy*2/3)))
valE = 1 # East-Dirichlet
resN = 0 # North - Neumann
resS = 0 # South - Neumann
resW = 0 # West - Neumann

# 2D Parabolic Boundary - Bottom - Initial conditios:
f0 = 0;

Numerical solution - finite difference method:

The numerical solution is given by the explicit form of our equation, found by applying the finite difference method.

In [2]:
# Memory allocation:
F = [[[0.0 for k in range(NT)] for j in range(NPy)] for i in range(NPx)] 
F = np.array(F)

# Initial conditions:
F[:,:,0] = f0

# Dirichlet boundary conditions (Are the same for each time step):
F[NPx-1,rangeE,:] = valE

# Neumann Boundary conditions (Are different at each time step, 
# so now we can only apply them to the initial time step).
F[0,:,0] = F[1,:,0]-dx*resW; #West (FDS)
F[:,0,0] = F[:,1,0]-dy*resS; #South (FDS)
F[:,NPy-1,0] = F[:,NPy-2,0]+dy*resN; #North (BDS)

# Integration:

for k in range(1,NT-1):
    f = F[:,:,k-1] # Auxiliary variable
    
    for j in range(1,NPy-1):
        for i in range(1,NPx-1):
            # Internal points:
            F[i,j,k] = f[i,j] - u*dt/2/dx*(f[i+1,j]-f[i-1,j])\
                              - v*dt/2/dy*(f[i,j+1]-f[i,j-1])\
                              + D*dt/dx**2*(f[i+1,j]+f[i-1,j]-2*f[i,j])\
                              + D*dt/dy**2*(f[i,j+1]+f[i,j-1]-2*f[i,j])
            
    # Neumann boundary conditions:
    F[0,:,k] = F[1,:,k]-dx*resW; #West (FDS)
    F[:,0,k] = F[:,1,k]-dy*resS; #South (FDS)
    F[:,NPy-1,k] = F[:,NPy-2,k]+dy*resN; #North (BDS)

Let's make a 2D animated plot of the result:

In [3]:
# Meshgrid:
X, Y = np.meshgrid(xspan, yspan)

# Figure:
fig, ax = plt.subplots()
cp = ax.contourf(X, Y, np.transpose(F[:,:,0]))

# Axes:
plt.xlabel('x [m]')
plt.ylabel('y [m]')

# Colorbar:
cbar = fig.colorbar(cp)

# Textbox:
string = 'f at time: %.2f [s]' %tspan[0]
txt = plt.text(Lx/2,1.05*Ly,string,size=14,ha="center", va="center")

# Initialize function:
def init():
    cp = ax.contourf(X, Y, np.transpose(F[:,:,0]))
    string = 'f at time: %.2f [s]' %tspan[0]
    txt.set_text(string)
    return cp, txt

# Update function:
def update(frame):
    cp = ax.contourf(X, Y, np.transpose(F[:,:,frame]))
    string = 'f at time: %.2f [s]' %tspan[frame]
    txt.set_text(string)
    return cp, txt

#Animation:
anim = animation.FuncAnimation(fig, update, init_func=init,\
                              frames = NT, interval = 100, blit=False)
# Close plot so it doesn't show up:
plt.close()

# Show video:
HTML(anim.to_html5_video())


Show how long did it take to execute the script:

In [4]:
stop = timeit.default_timer()
execution_time = stop-start
print('Script executed in {:2.2f} [s]'.format(execution_time)) 

Script executed in 91.23 [s]
